### Initialize the GEE envrionment
___


In [1]:
import ee
#ee.Authenticate()
ee.Initialize()

This will make Google EE ready to use. Run the following codes to test if you can see a map. 

In [2]:
import geemap
Map = geemap.Map(center=[39.027884,-84.137946], zoom=12)
Map.add_basemap('SATELLITE') # Add Google Map
Map

Map(center=[39.027884, -84.137946], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

In [ ]:
#Map.setCenter(-90.162, 29.8597, 12)   # New Orleans, USA , change the coordinate to your study area

## Search for sentinel images with given coordinate and time
A time delta is set to 2 days by default. It relaxes the search window from 1 day to $1\pm 2 days$

In [3]:
#search parameters are set here
from datetime import date, timedelta
pnt_date = date(2019, 11, 4)
lat =  39.027884
lon = -84.137946
delta = timedelta(days = 10)

In [4]:
from sentinel_search import sentinel2Availability, ImageExist
p1 = ee.Geometry.Point([lon, lat])
#  search sentinel first
img = sentinel2Availability(ee,pnt_date, lat, lon, delta, True)
bands = ['B1','B2','B3','B4','B5','B7']
if ImageExist(img) == 'true':
    ac_date = ee.Date(img.get('system:time_start')).format('MM/dd/YYYY');
    print('Found an ' + ' image for date:', pnt_date, ' at date:', ac_date.getInfo())
    img1 = img.select(['B2','B3','B4','B8A','B11','B12'],['B1','B2','B3','B4','B5','B7'])
    
    #the band names are defined to match Landsat TM bands
    vis = {'bands': ['B4', 'B3', 'B2'],'min': [0, 0, 0], 'max': [1000,1000, 1000]} #reflectance * 10000
    Map.addLayer(img1, vis,'Sentinel')
else:
    print('No image was found, try another date')
 

Found an  image for date: 2019-11-04  at date: 11/01/2019


## use featurecollection to create sample data points.
The feature collections can be sampled from the image if the ground sample points are supplied


In [7]:
p1 = ee.Geometry.Point(-84.138042,39.035259)
p2 = ee.Geometry.Point(-84.133715,39.035095)
p3 = ee.Geometry.Point(-84.142131,39.031263)
p4 = ee.Geometry.Point(-84.137765,39.031283)
p5 = ee.Geometry.Point(-84.133436,39.031577)
p6 = ee.Geometry.Point(-84.137946,39.027884)
p7 = ee.Geometry.Point(-84.133342,39.027865)
p8 = ee.Geometry.Point(-84.151048,39.016645)

# Use these bands for prediction.

# Manually created points for the purpose of testing

Points = ee.FeatureCollection([
  ee.Feature(p1, {'Chl_a': 4.977778}),
    ee.Feature(p2, {'Chl_a': 5.072131}),
    ee.Feature(p3, {'Chl_a': 4.843056}),
    ee.Feature(p4, {'Chl_a': 5.237681}),
    ee.Feature(p5, {'Chl_a': 5.336842}),
    ee.Feature(p6, {'Chl_a': 5.508696}),
    ee.Feature(p7, {'Chl_a': 5.622078}),
    ee.Feature(p8, {'Chl_a': 10.368889})
])
#you can use your own feature class uploaded as Assets as samples:
Points = ee.FeatureCollection("users/leiwanglsu/water_quality/harsha")
training = img1.sampleRegions(**{
  'collection': Points,
  'properties': ['Chl_a'],
  'scale': 15
})
# Train the classifier.
svm_trained = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'svmType': 'EPSILON_SVR',
  'gamma': 0.3,
  'cost': 1
}).setOutputMode('REGRESSION').train(training, 'Chl_a', bands)

rf_trained = ee.Classifier.smileRandomForest(**{
    'numberOfTrees':100}).setOutputMode('REGRESSION').train(training, 'Chl_a', bands)
# Classify the image.
classified_rf = img1.classify(rf_trained)
classified_svm = img1.classify(svm_trained)

In [8]:
# Display the classification result and the input image.

Map.addLayer(classified_rf,
             {'min': 1, 'max': 10, 'palette': ['red', 'green']},
             'Chl_a_rf')
Map.addLayer(classified_svm,
             {'min': 1, 'max': 5, 'palette': ['red', 'green']},
             'Chl_a_svm')
Map.addLayer(Points,name='Samples')